<h1><center>Universidad de los Andes ‐ Ciencia de datos aplicada
 </center>
    <h1><center>Sprint 2: Dataset Udemy </center></h1>
    
<center>Ana María Montes Franco - 202010396 </center>
<center>Nicolas Rozo Bedoya - 2021 </center>
<center>Mateo Zapata Lopez- 2021 </center>

## Contexto del problema 

Udemy es una plataforma digital (MOOC-> Massive online open course) creada en el año 2009 que tiene como objetivo ofrecer cursos en línea gratuitos y pagos, que ofrencen clases de programación, finanzas, diseño gráfico, música, entre otros. La misión de Udemy es conectar a los estudiantes con los mejores instructores y ayudar a las personas alcanzar sus metas y perseguir sus sueños. Actualmente tiene más de 50 millones de estudiantes y 57.000 instructores enseñando 150.000 cursos en 65 lenguajes. Algunos de los cursos estan acreditados y generan certificaciones. 

Udemy permite crear y vender el contenido de una persona, que se le domina como el instructor. Los estudiantes pueden comprar el curso y el instructor recibe un porcentaje de la compra. El problema esta en que Udemy no cuenta con un estándar en los cursos ofrecidos, generando alta variabilidad en la estructura de los mismo. Adicionalmente, la gran cantidad y variedad de plataformas, tanto privadas como open source, evidencia que Udemy no ofrece un elemento diferenciador. 

Este dataset continene 3.682 datos de los cursos correspondientes a 4 categorias, el estudio se realiza para datos desde el 2011 hasta el 2017. 
> ### Caracteristicas del Dataset:
>> 1. course_id: Identificación del curso 
>> 2. course_title: Titulo del curso 
>> 3. url: URL del curso 
>> 4. is_paid: True si el curso es pago, false si el curso es gratuito 
>> 5. price: Precio del curso 
>> 6. num_subscribers: Numero de subscriptores del curso 
>> 7. num_lectures: Numero de lecturas del curso 
>> 8. num_reviews: Numeri de reviews del curso 	
>> 10. level: Nivel de dificultad del curso 
>> 11. content_duration: Duración de los materiales del curso 
>> 12. published_timestamp: Fecha de publicación del curso 
>> 13. subject: Categoria del curso 

In [1]:
# Librerias 
import numpy as np
#Librería de manejo de datos
import pandas as pd
#Para regresión 
from sklearn import linear_model as lm
from sklearn.linear_model import LinearRegression
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Grid search para obtener los mejores hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Preprocesar
from sklearn import preprocessing
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Para sacar un reporte estadístico que podemos usar para determinar las importancia de las variables explicativas.
import statsmodels.api as sm 
# Librerías para gráficos
import matplotlib.pyplot as plt
import seaborn as sns
#Pandas profiling 
from pandas_profiling import ProfileReport
import plotly.express as px
# Libreria para comparacion de distribucion entre dos conjuntos
from scipy.stats import ks_2samp

In [2]:
# Lectura del dataset 
ucourses=pd.read_csv('udemy_courses.csv',encoding='latin-1',sep=',')
ucourses.head(5)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


In [3]:
#Dimensiones del dataset
ucourses.shape 

(3678, 12)

In [4]:
# Información sobre el tipo de cada columna del dataset 
ucourses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3678 non-null   int64  
 1   course_title         3678 non-null   object 
 2   url                  3678 non-null   object 
 3   is_paid              3678 non-null   bool   
 4   price                3678 non-null   int64  
 5   num_subscribers      3678 non-null   int64  
 6   num_reviews          3678 non-null   int64  
 7   num_lectures         3678 non-null   int64  
 8   level                3678 non-null   object 
 9   content_duration     3678 non-null   float64
 10  published_timestamp  3678 non-null   object 
 11  subject              3678 non-null   object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 319.8+ KB


In [5]:
#limpieza de datos 
# 1. Comprobar las ausencias de datos por variable. 
ucourses.isna().sum()/len(ucourses)

course_id              0.0
course_title           0.0
url                    0.0
is_paid                0.0
price                  0.0
num_subscribers        0.0
num_reviews            0.0
num_lectures           0.0
level                  0.0
content_duration       0.0
published_timestamp    0.0
subject                0.0
dtype: float64

El dataset no cuenta con datos faltantes lo que facilita la etapa de preparación de los datos 

In [6]:
#Con el fin de conocer en primera instancia los datos, se realizar el perfilamiento
profile = ProfileReport(ucourses, title="Perfilamiento de datos")

In [ ]:
profile

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

las gráficas suministradas en los histogramas suministrados por el profile realizado se encuentra que la mayoría de los cursos tienen una duración entre 0-5 horas, que usualmente hay entre 1 y 50 lecturas por curso, que los cursos suelen tener pocos reviews, la mayoría de los cursos están en el mismo rango de suscriptores, hay cursos para los cuales es muy marcado que se encuentran en el trending topic y para finalizar suponiendo que los precios se encuentran en dólares, el rango esta entre 0 y 250 dólares, mostrando como el precio más común es $25. 

In [ ]:
#Limpieza de datos 
#2. Eliminar duplicados 

## El profiling muestra que se tienen 6 datos duplicados
#Gracias al entendimiento de los datos se identifica que se deben eliminar instancias duplicadas correspondiente 
#al URL dado que cada curso cuenta con uno propio  
ucourses["url"].nunique()

In [ ]:
# El resultado anterior de 3672 respecto al original de 3678 nos permite confirmar que estos 6 datos duplicados corresponden
# al url del curso por esta razón se eliminan 
ucourses.drop_duplicates("url", inplace=True)

In [ ]:
#Verificar que se esos 6 corresponden al url 
ucourses.shape

In [ ]:
#Preparación de datos-> Trasnformación de variables 
# La ariable tiempo viene en "object format" y debe ser remplazada por el formado datatime, adicionalmente luego de extra 
# transformación se extrae el dato de interes correspondiente al año y se crea una nueva columna 

ucourses['year'] = pd.DatetimeIndex(ucourses['published_timestamp']).year

In [ ]:
#Preparación de datos-> Enriquecimiento del dataset 
# Nueva columna en formato datatime que contiene el año de publicación del curso 
ucourses['year']

In [ ]:
#Preparación de datos-> Selección de variables significativas para el analisis 
ucourses.corr()

In [ ]:
# Como se pudo analizar en el perfiladmiento de los datos hay 3 variables que no aportan información significativa 
# Por este motivo seran eliminadas del dataframe 
ucourses=ucourses.drop(["published_timestamp","url","course_id"],axis=1)

In [ ]:
#Preparación de datos-> Enriquecimiento del dataset 
# Se construye una nueva columna en la cual se calcula cual es la gancian por subscriptor cuando el curso es pago 
ucourses["ganancia"]=ucourses["price"]*ucourses["num_subscribers"]

In [ ]:
#Preparación de datos-> Construcción de un nuevo dataframe

# Columnadas nuevo dataframe 
ucourses.head(1)

In [ ]:
# Analitica descriptiva de las principales variables numericas 
ucourses.describe()

A nivel general se destaca que la media del precio es 66,10 la cantidad de subscriptores 3190, de reviews 156 y la duración del contenido es de 4 horas aproximadamente.  

Con la información que se tiene hasta este punto y teniendo presente que algo importante para los instructores de udemy es que cursos deben actualizar o no su contenido se profundizara el analisis exploratorio en los cursos más populares dentro del amplio portafolios, no con el fin de brindar un mejor servicio y jerarquizar el trabajo. 

In [ ]:
#Preparación de datos-> Transformación de las variables categoricas 
#Cantidad de cursos que ofrece Udmey por categoria. 
ucourses['subject'].value_counts()

In [ ]:
# Exploración de datos 

#Cantidad de cursos por año 
ucourses['year'].value_counts()

In [ ]:
# Exploración de datos -> Analisis bivariado 

plot_data = ucourses.groupby(['year', 'subject'])['year'].count().to_frame()
plot_data['subject'] = [ x[1] for x in plot_data.index.tolist()]
plot_data.index = [x[0] for x in plot_data.index.tolist()]
plot_data['time'] = plot_data.index
plot_data = plot_data.reset_index(drop=True)
sns.catplot(x = 'time', y="year", hue="subject",palette = sns.color_palette("magma"), kind="point",data=plot_data)
plt.show()

El grafico bivariado permite observar con mayor claridad la distribución de las categorías, ver las tendencias ya que a diferencia de los diagramas del perfilamiento en este es notable la diferencia entre cada una y los picos que a tenido la serie en los 7 años de análisis, destacando sobre todo las dos primeras categorías

In [ ]:
 # Exploración de datos -> Analisis multivariado

df1 = ucourses.groupby('year')[['num_reviews','num_lectures']].sum().reset_index()
df1 = df1[df1.year != 2017]
### Se analiso que para el año 2017 no se tiene toda la información 
df1 = df1.set_index('year')
df1.plot.bar(rot=0)

Este diagrama de barras multivariado permite observar la comparación del comportamiento del número de reviews respecto a el numero de lecturas para los años de análisis. 

In [ ]:
ucourses["level"].value_counts()

In [ ]:
# Exploración de datos -> EXTRAER EL TOP 10 DE LOS CURSOS MÁS POPULARES DE UDEMY 

# Para realizar estas graficas construimos un nuevo dataframe  
df = ucourses.groupby(['is_paid',"subject","level", "course_title",'content_duration'])[['num_subscribers','num_reviews','num_lectures']].sum().reset_index()
df

In [ ]:
# Grafica top 10 de los cursos con más subscriptores 

top_course = df.sort_values(by='num_subscribers', ascending=False)[:10]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course.course_title, x=top_course.num_subscribers)
plt.xticks()
plt.xlabel('num_subscribers')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses')
plt.show()

In [ ]:
# Exploración de datos -> Cantidad de cursos pagos y gratutios en el top respecto al # de subscriptores 

pdf= top_course[['is_paid', 'subject']].value_counts().reset_index()
pdf

In [ ]:
#Top 10 Most Popular Courses Reviews by content duration

top_course2 = df.sort_values(by='content_duration', ascending=False)[:10]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course2.course_title, x=top_course2.content_duration)
plt.xticks()
plt.xlabel('content_duration')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses content duration')
plt.show()


In [ ]:
#Top 10 Most Popular Courses Reviews by num_reviews

top_course2 = df.sort_values(by='num_reviews', ascending=False)[:10]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course2.course_title, x=top_course2.num_reviews)
plt.xticks()
plt.xlabel('num reviews')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses Reviews')
plt.show()


In [ ]:
#Top 10 Most Popular Courses Reviews by num_lectures

top_course2 = df.sort_values(by='num_lectures', ascending=False)[:10]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course2.course_title, x=top_course2.num_lectures)
plt.xticks()
plt.xlabel('num lectures')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses Lectures')
plt.show()

In [ ]:
# Exploración de datos -> Cantidad de cursos pagos y gratutios en el top respecto a la duración del contenido  
pdf2= top_course2[['is_paid', 'subject']].value_counts().reset_index()
pdf2

In [ ]:
# Exploración de datos -> Variable de es pago o no 

#Diagram de barras con proporcion 
plt.figure(figsize=(10,5))
sns.countplot(x=ucourses["is_paid"])
plt.title("Free and Paid Courses", size=20)
plt.xlabel("is_paid", size = 15)
plt.ylabel("count", size=15)
plt.tight_layout()
plt.show()

In [ ]:
# Exploración de datos -> Analisis multivariado
## Analisis de la distribución de los precios segun la categoria 

fig = plt.figure(figsize=(15,8),)
ax=sns.kdeplot(ucourses.loc[(ucourses['subject'] == 'Business Finance'),'price'] , color='yellow',shade=True,label='Business Finance')
ax=sns.kdeplot(ucourses.loc[(ucourses['subject'] == 'Graphic Design'),'price'] , color='black',shade=True, label='Graphic Design')
ax=sns.kdeplot(ucourses.loc[(ucourses['subject'] == 'Musical Instruments'),'price'] , color='red',shade=True,label='Musical Instruments')
ax=sns.kdeplot(ucourses.loc[(ucourses['subject'] == 'Web Development'),'price'] , color='green',shade=True,label='Web Development')
plt.title('Price Distribution', fontsize = 25, pad = 40)
plt.ylabel("Frequency of Price", fontsize = 15, labelpad = 20)
plt.xlabel("Price", fontsize = 15, labelpad = 20);

Esta grafica permite confirmar muchos de los análisis realizados respecto al nivel principiante dado que los cursos de mayor valor correspondientes a nivel experto son los menos comunes y se ven nuevamente los picos observados en el perfilamiento, destacando que la categoría más popular correspondiente a Web development en color verde no es la que tiene los precios más altos. 

In [ ]:
# Exploración de datos -> Analisis multivariado 
# Box plot 

fig=px.box(ucourses,x="content_duration",y="subject",orientation="h",color="is_paid",
          title="Content duration according to subject and type of course",
          color_discrete_sequence=["#03cffc",'#eb03fc'])
fig.update_xaxes(title="content duration")
fig.update_yaxes(title="course subject")
fig.show()


Es evidente como cuando el contenido es gratis la duracion es menor 

In [ ]:
 # Exploración de datos -> Analisis multivariado
    
plt.rcParams['figure.figsize'] = (30, 50)
sns.catplot(x="level", kind="count",hue ='subject', data=ucourses, col='is_paid');
plt.show()

In [ ]:
# Exploración de datos -> Analisis multivariado

#Conteo de cursos gratis por categoria 

Free_courses=ucourses[ucourses['price']==0].groupby(['subject']).count()
Free_courses


La categoria que más cuenta con cursos gratuitos es Web Development con una cantidad de 133 cursos, seguido de Business Finance con 96 

In [ ]:
#### Cursos pagos por categoria 

Paid_courses=ucourses[ucourses['price']!='Free'].groupby(['subject']).count()
Paid_courses

In [ ]:
# Exploración de datos -> Analisis multivariado

fig_dims = (15, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x="subject",y="num_lectures",data=df,ax=ax,palette="cubehelix",estimator=np.mean,hue=df.level)

In [ ]:
## Trasnformación variable precio

####### Label encoding 
def labcat(price):
    if price >=0 and price<20: #(Economico)
        return 1
    if price >=20 and price<45: #(Estandar)
        return 2
    if price >=45 and price<90: #(Premium)
        return 3
    else: 
        return 4                #(Royal)

In [ ]:
Data1= ucourses
Data1["price"]=Data1["price"].apply(labcat)

In [ ]:
# Datos codificados 
Data1["price"]

In [ ]:
# Segun lo encontrado en el analisis posterior se quiere conocer de manera detallada la correlacion de las variables que más se destacaron 
## Para la seleccion de variables volver a mirar el diagrama de correlación asi ya este en el profiling 

cor = ucourses.corr() 
sns.heatmap(cor, square = True) 

In [ ]:
# Otro formato del diagrama de correlación que permite observar con mayor detalle lo obtenido 
sns.heatmap(ucourses.corr(), cmap="coolwarm", annot=True);

El diagrama de correlación permite observa que no existe una fuerte correlación entre las variables, pero si una moderada entre número de suscriptores, reviews y lectures. Con esta relación identificada se plantean los siguientes modelos de predicción:  

## Modelos de predicción 
Debido a los valores de correlacion se realizan solamente regresiones lineales simples 
1. Modelo 1: Se busca predecir el numero de reviews que se tendra segun un numero de subscriptores esto teniendo en cuenta el flujo ya que solo se puede obtener un review al estar subscrito 

2. Modelo 2: Se busca predecir la duración del contenido segun la cantidad de lecturas

## Modelo 1

In [ ]:
# Se selecciona la variable objetivo, en este caso "numero de reviews".
Y=ucourses['num_reviews']
#Variables predictoras 
X=ucourses["num_subscribers"].values.reshape(-1, 1)

### Separación de los datos en conjuntos de prueba y entrenamiento y verificación que las distribuciones de ambos conjuntos son similares.

In [ ]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=0)

In [ ]:
##### Distribución luego del split 

print("Number of instances in training set: ", len(X_train))
print("Number of instances in testing set: ", len(X_test))

 Se realiza la validación de las disitrubcion de ambos conjuntos usando el test de Kolmogorov.
 Se valida que la distribucion de los conjuntos de datos cumpla con la condición que su distribución sea
 uniforme, exponencial, posion o normal usando pandas profiling


In [ ]:
dfY_train = pd.DataFrame(Y_train)
dfY_test = pd.DataFrame(Y_test)
dfX_train = pd.DataFrame(X_train)
dfX_test = pd.DataFrame(X_test)

In [ ]:
perfilamientoY_train=ProfileReport(dfY_train)
perfilamientoY_train

In [ ]:
perfilamientodfY_test=ProfileReport(dfY_test)
perfilamientodfY_test

In [ ]:
perfilamientodfX_test=ProfileReport(dfX_test)
perfilamientodfX_test

In [ ]:
perfilamientoY_train=ProfileReport(dfY_train)
perfilamientoY_train

Teniendo en cuenta que la distribución del conjunto de datos es de tipo exponencial se puede realizar el test de 
Kolmogorov-Smirnov

In [ ]:
ks_2samp(X_train[:, 0], X_test[:, 0])

In [ ]:
ks_2samp(Y_train, Y_test)

La hipótesis nula es que las dos distribuciones son idénticas. Si el estadístico K-S es pequeño o el valor p es alto (mayor que el nivel de significancia de 5%), en este caso se acepta la hipótesis de que las distribuciones de las dos muestras son similares. Sí el valor p es bajo se rechazaria la hipótesis nula.

In [ ]:
# 3. Construcción del modelo 

# Primero se crea el objeto para construir el modelo
modelo_regresion = LinearRegression()
# Podemos verificar que lo hemos construido.
modelo_regresion

In [ ]:
# Ajustar el modelo con los datos de entrenamiento
modelo_regresion = modelo_regresion.fit(X_train,Y_train)

In [ ]:
# Predicciones del modelo sobre el conjunto de entrenamiento.
y_pred = modelo_regresion.predict(X_train)

In [ ]:
# 4. Evaluación del modelo 

# Que tan bien se desempeña el modelo, como se ajustan los datos
# Mean-Squared-Error(MSE). Error medio cuadrático  ---> Varianza (Se busca que sea lo menor posible)
# Mean-Absolute-Error(MAE). Error absoluto medio   ---> Desviación absoluta promedio 
# R² or Coeficiente de determinación ---> Se busca que sea cercano a 1 

# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print("MSE: %.2f" % mean_squared_error(Y_train, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_train, y_pred))
#print('R²: %.2f' % r2_score(Y_train, y_pred))

## un R2 cercano a 1 es overt y cercano a 0 under
## Si le saco la raiz al MSE puedo decir que en promedio me equivoco o varia entre el valor que me de 

In [ ]:
# 5. Interpretacion del modelo

# Ajustar el modelo para ver el reporte
model = sm.OLS(Y, X).fit() ## sm.OLS(output, input)
# Mostrar las estadísticas del modelo
model.summary()

In [ ]:
## Para sacar resultados de la tabla 
print('coefficient of determination:', model.rsquared)
print('adjusted coefficient of determination:', model.rsquared_adj)
print('regression coefficients:', model.params) ## B0,b1,b2

In [ ]:
# Se saca el coeficiente y el intercepto del modelo
print(modelo_regresion.intercept_)
print(modelo_regresion.coef_)

### Modelo 2

In [ ]:
# Se selecciona la variable objetivo, en este caso "num_subscribers".
Y2=ucourses[["num_subscribers"]]
#Variables predictoras 
X2=ucourses[["price","content_duration","num_lectures"]]

In [ ]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split( X2,Y2, test_size=0.2, random_state=0)

In [ ]:
##### Distribución luego del split 
print("Number of instances in training set: ", len(X_train))
print("Number of instances in testing set: ", len(X_test))

In [ ]:
# 3. Construcción del modelo 

# Primero se crea el objeto para construir el modelo
modelo_regresion = LinearRegression()
# Podemos verificar que lo hemos construido.
modelo_regresion

In [ ]:
# Ajustar el modelo con los datos de entrenamiento
modelo_regresion.fit(X_train,Y_train)

In [ ]:
# Predicciones del modelo sobre el conjunto de entrenamiento.
y_pred = modelo_regresion.predict(X_train)

In [ ]:
# 4. Evaluación del modelo 

# Que tan bien se desempeña el modelo, como se ajustan los datos
# Mean-Squared-Error(MSE). Error medio cuadrático  ---> Varianza (Se busca que sea lo menor posible)
# Mean-Absolute-Error(MAE). Error absoluto medio   ---> Desviación absoluta promedio 
# R² or Coeficiente de determinación ---> Se busca que sea cercano a 1 

# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print("MSE: %.2f" % mean_squared_error(Y_train, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_train, y_pred))
print('R²: %.2f' % r2_score(Y_train, y_pred))

## un R2 cercano a 1 es overt y cercano a 0 under
## Si le saco la raiz al MSE puedo decir que en promedio me equivoco o varia entre el valor que me de 

In [ ]:
# 5. Interpretacion del modelo

# Ajustar el modelo para ver el reporte
model = sm.OLS(Y2, X2).fit() ## sm.OLS(output, input)
# Mostrar las estadísticas del modelo
model.summary()

In [ ]:
## Para sacar resultados de la tabla 
print('coefficient of determination:', model.rsquared)
print('adjusted coefficient of determination:', model.rsquared_adj)
print('regression coefficients:', model.params) ## B0,b1,b2

In [ ]:
# Se saca los coeficiente y el intercepto del modelo
print(modelo_regresion.intercept_)
print(modelo_regresion.coef_)

## OTRA FORMA DE IMPLEMENTAR LA REGRESIÓN LINEAL MULTIPLE 

In [ ]:
#Multiple Linear Regression
inputDF = ucourses[["price","content_duration"]]
outputDF = ucourses[["num_subscribers"]]

model = lm.LinearRegression()
results = model.fit(inputDF,outputDF)

print(model.intercept_, model.coef_)

In [ ]:
outputDF_predict = model.predict(inputDF)
print(np.hstack((inputDF,outputDF, outputDF_predict)))

In [ ]:
#Linear Regression representation using scatter plot
plt.scatter(inputDF.content_duration,outputDF)
plt.plot(inputDF.content_duration, outputDF_predict, color="blue")
plt.show()

Kevin> as as predictor variables are clustered in between 0 and 5000,overall outcome of the linear regression seemed toe be influenced.

Las variables predictivas estan concentradas entre 0 y 5000